# OmegaZero - ChessGame Exploration

This notebook demonstrates the ChessGame class from the OmegaZero chess engine.

**Purpose:** Explore the game environment and understand the board representation for neural network training.

## Setup and Imports

In [ ]:
import numpy as np
import chess
import chess.svg
from IPython.display import SVG, display

from chess_game import ChessGame

## Basic Usage

In [ ]:
# Create a new game at starting position
game = ChessGame()
print(f"Game state (FEN): {game.get_state()}")
print(f"Is game over? {game.is_game_over()}")

## Board Visualization

Display the chess board using SVG (works in Jupyter Lab).

In [ ]:
def show_board(game: ChessGame):
    """Display chess board as SVG."""
    svg_data = chess.svg.board(game.board, size=400)
    display(SVG(svg_data))

show_board(game)

## Legal Moves

In [ ]:
# Get legal moves
legal_moves = game.get_legal_moves()
print(f"Number of legal moves: {len(legal_moves)}")
print(f"\nFirst 10 legal moves (UCI notation):")
for i, move in enumerate(legal_moves[:10]):
    print(f"  {i+1}. {move.uci()}")

## Making Moves

In [ ]:
# Make some moves (Scholar's Mate setup)
moves = ["e2e4", "e7e5", "f1c4", "b8c6", "d1h5", "g8f6", "h5f7"]

for move_uci in moves:
    move = chess.Move.from_uci(move_uci)
    game.make_move(move)
    print(f"Made move: {move_uci}")

print(f"\nIs game over? {game.is_game_over()}")
if game.is_game_over():
    result = game.get_result()
    print(f"Result: {result} (from current player perspective)")

show_board(game)

## Canonical Board Representation

This is the neural network input format - always from current player's perspective.

In [ ]:
# Start fresh game
game = ChessGame()

# Get canonical board
board_tensor = game.get_canonical_board()

print(f"Board tensor shape: {board_tensor.shape}")
print(f"Board tensor dtype: {board_tensor.dtype}")
print(f"\nTensor planes:")
print(f"  Planes 0-5: Current player's pieces (P, N, B, R, Q, K)")
print(f"  Planes 6-11: Opponent's pieces (P, N, B, R, Q, K)")
print(f"  Plane 12: Repetition count")
print(f"  Plane 13: En passant square")

In [ ]:
# Examine specific planes
def show_plane(board_tensor: np.ndarray, plane_idx: int, title: str):
    """Display a single plane from the board tensor."""
    print(f"\n{title} (Plane {plane_idx}):")
    plane = board_tensor[:, :, plane_idx]
    # Print with rank labels (8 to 1)
    print("    a  b  c  d  e  f  g  h")
    for rank in range(7, -1, -1):
        print(f"{rank+1}  ", end="")
        for file in range(8):
            value = int(plane[rank, file])
            print(f" {value} ", end="")
        print(f" {rank+1}")
    print("    a  b  c  d  e  f  g  h")

# Show white pawns (current player's pawns)
show_plane(board_tensor, 0, "White Pawns")

# Show white king
show_plane(board_tensor, 5, "White King")

## Move Encoding

Demonstration of converting between chess moves and neural network policy indices.

In [ ]:
# Get a legal move
move = chess.Move.from_uci("e2e4")

# Convert to policy index
move_idx = game.get_move_index(move)
print(f"Move {move.uci()} -> Policy index {move_idx}")

# Convert back
recovered_move = game.get_move_from_index(move_idx)
print(f"Policy index {move_idx} -> Move {recovered_move.uci()}")

# Verify round-trip
assert move == recovered_move, "Move encoding round-trip failed!"
print("\n✅ Move encoding/decoding works correctly!")

## Legal Moves Mask

Binary mask for valid moves (used by neural network to avoid selecting illegal moves).

In [ ]:
# Get legal moves mask
legal_mask = game.get_legal_moves_mask()

print(f"Legal moves mask shape: {legal_mask.shape}")
print(f"Legal moves mask dtype: {legal_mask.dtype}")
print(f"Number of legal moves: {int(legal_mask.sum())}")
print(f"Number of illegal moves: {int((1 - legal_mask).sum())}")

# Show some legal move indices
legal_indices = np.where(legal_mask == 1)[0]
print(f"\nFirst 10 legal move indices: {legal_indices[:10]}")

## Game Cloning

Essential for MCTS - create independent copies of game state.

In [ ]:
# Create original game
original = ChessGame()
print(f"Original state: {original.get_state()}")

# Clone it
cloned = original.clone()
print(f"Cloned state:   {cloned.get_state()}")

# Make move in clone
cloned.make_move(chess.Move.from_uci("e2e4"))
print(f"\nAfter move in clone:")
print(f"Original state: {original.get_state()}")
print(f"Cloned state:   {cloned.get_state()}")

# Verify independence
assert original.get_state() != cloned.get_state(), "Clone not independent!"
print("\n✅ Cloning creates independent game states!")

## Custom Positions (FEN)

Load specific positions for testing.

In [ ]:
# Create game from FEN (endgame position)
fen = "8/5k2/8/8/8/8/3K4/8 w - - 0 1"  # King and pawn endgame
endgame = ChessGame(fen)

print(f"Position: {fen}")
print(f"Legal moves: {len(endgame.get_legal_moves())}")

show_board(endgame)

## Integration Test: Full Game Flow

Simulate what will happen during self-play training.

In [ ]:
def simulate_game_step(game: ChessGame) -> dict:
    """
    Simulate one step of what happens during self-play.
    
    Returns:
        Dictionary with neural network inputs/outputs
    """
    # Get board representation (NN input)
    board_tensor = game.get_canonical_board()
    
    # Get legal moves mask
    legal_mask = game.get_legal_moves_mask()
    
    # Simulate neural network policy output (uniform over legal moves)
    policy = legal_mask / legal_mask.sum()
    
    # Simulate neural network value output (random for demo)
    value = np.random.uniform(-0.5, 0.5)
    
    return {
        'board': board_tensor,
        'legal_mask': legal_mask,
        'policy': policy,
        'value': value
    }

# Run simulation
game = ChessGame()
move_count = 0

while not game.is_game_over() and move_count < 10:
    # Simulate NN prediction
    result = simulate_game_step(game)
    
    print(f"\nMove {move_count + 1}:")
    print(f"  Board shape: {result['board'].shape}")
    print(f"  Legal moves: {int(result['legal_mask'].sum())}")
    print(f"  Policy sum: {result['policy'].sum():.4f}")
    print(f"  Value prediction: {result['value']:.4f}")
    
    # Make random legal move
    legal_moves = game.get_legal_moves()
    move = np.random.choice(legal_moves)
    game.make_move(move)
    print(f"  Executed move: {move.uci()}")
    
    move_count += 1

print("\n✅ Game flow simulation complete!")
show_board(game)

## Summary

This notebook demonstrated:

1. ✅ **ChessGame initialization** - Starting position and custom FEN
2. ✅ **Board visualization** - SVG rendering in Jupyter
3. ✅ **Legal moves** - Generation and validation
4. ✅ **Move execution** - Making moves and checking game state
5. ✅ **Canonical board** - Neural network input format (8, 8, 14)
6. ✅ **Move encoding** - Converting between chess.Move and policy indices
7. ✅ **Legal moves mask** - Binary mask for valid moves
8. ✅ **Game cloning** - Independent copies for MCTS
9. ✅ **Integration flow** - Simulating self-play training loop

**Next Steps:**
- Neural Network (ChessNN) - Policy and value predictions
- MCTS - Tree search using neural network
- Self-Play - Generate training games
- Training Pipeline - Learn from self-play data